#### Table Replication  
Use predefined SQL table copies to partion and cluster new tables

In [1]:
import bq, config, sfa
from google.cloud import bigquery

In [10]:
from importlib import reload

In [2]:
project = "tpc-benchmarking-9432"

In [3]:
fp_schema = config.fp_schema + config.sep + "ds_01A.sql"

In [4]:
source_dataset = "ds_1GB_01"
destination_dataset = "ds_1GB_03"

In [5]:
sf = sfa.SFTPC(test="ds",
               scale=100,
               cid="",
               warehouse="TEST9000", 
               desc="dev_clone_test_1",
               verbose=True,
               verbose_query=False)

In [8]:
sf.database = "DS_100GB"

In [23]:
reload(sfa)
sf = sfa.SFTPC(test="ds",
               scale=100,
               cid="0X",
               warehouse="TEST9000", 
               desc="dev_clone_test_1",
               verbose=True,
               verbose_query=False)
sf.database = "DS_100GB_0X"
sf.connect()

query_result = sf.copy("DS_100GB_01")
print(query_result.sfqid)

sf.close()

Snowflake configuration
Username: colin
Account:  wja13212

019468bf-026f-2ad5-0009-c083000469da


In [16]:
query_result

In [20]:
query_result.sfqid

'01946798-0291-a6c8-0009-c08300048366'

In [ ]:
def create_table_remix(fp_schema, source, destination, dot=False):
    
    if dot:
        source = source + "."
        destination = destination + "."
    
    with open(fp_schema, "r") as f:
        query_text = f.read()
    query_text = query_text.replace("_source_table.", source)
    query_text = query_text.replace("_destination_table.", destination)
    return query_text

In [ ]:
query_text = create_table_remix(fp_schema=fp_schema,
                                source=source_dataset,
                                destination=destination_dataset,
                                dot=True)

In [ ]:
print(query_text[0:200])

In [ ]:
query_text = create_table_remix(fp_schema=fp_schema,
                                source="",
                                destination="",
                                dot=False)

In [ ]:
print(query_text[0:200])

In [ ]:
bq.create_dataset(destination_dataset)

In [ ]:
%%time
query_job = bq.query(query_text=query_text,
                     project=project,
                     dataset=destination_dataset,
                     dry_run=False,
                     use_cache=False)
query_job.result()